In [1]:
# First we set the corresponding working directory

# path for Victor
# setwd("Documents/workspace/codon_projects/codon_suberin/Rscripts")

# path for Alejandra
setwd("/Users/ale/Documents/thesis_codon_bias/Testing_the_TEH/Rscripts")

# Installing all necessary packages
# install.packages("RColorBrewer")
# install.packages("seqinr")
# install.packages("VGAM")
# install.packages("doSNOW")
# install.packages("coda")
# install.packages("EMCluster")
# install.packages("Biostrings")
# if (!requireNamespace("BiocManager", quietly = TRUE))
#   install.packages("BiocManager")
# # Need 3.14 with R version 4.1
# BiocManager::install(version = "3.14")
# BiocManager::install(c("GeneGA"))
# BiocManager::install("sscu")
# install.packages("bioseq")
# install.packages("GeneGA")
# install.packages("AnaCoDa")

# load necessary pacakges
library("RColorBrewer")
library("seqinr")
library("VGAM")
library("doSNOW")
library("coda")
library("EMCluster")
library("Biostrings")
library("bioseq")
library("GeneGA")
library("AnaCoDa")

Warning message:
"le package 'RColorBrewer' a 'et'e compil'e avec la version R 4.1.2"
Warning message:
"le package 'seqinr' a 'et'e compil'e avec la version R 4.1.2"
Warning message:
"le package 'VGAM' a 'et'e compil'e avec la version R 4.1.2"
Le chargement a n'ecessit'e le package : stats4

Le chargement a n'ecessit'e le package : splines

Warning message:
"le package 'doSNOW' a 'et'e compil'e avec la version R 4.1.2"
Le chargement a n'ecessit'e le package : foreach

Warning message:
"le package 'foreach' a 'et'e compil'e avec la version R 4.1.2"
Le chargement a n'ecessit'e le package : iterators

Warning message:
"le package 'iterators' a 'et'e compil'e avec la version R 4.1.2"
Le chargement a n'ecessit'e le package : snow


Attachement du package : 'coda'


L'objet suivant est masqu'e depuis 'package:VGAM':

    nvar


Warning message:
"le package 'EMCluster' a 'et'e compil'e avec la version R 4.1.2"
Le chargement a n'ecessit'e le package : MASS

Warning message:
"le package 'MASS' 

In [2]:
# Read the nonsense error rates file
# Nonsense error rates obtained from FONSE model are stored in file "nonsense_error_rates_fonse.csv" under "$selection"
ecoli_nonsense_error_data = read.csv(file = "/Users/ale/Documents/thesis_codon_bias/Testing_the_TEH/Rscripts/nonsense_error_rates_fonse.csv", header = TRUE)

In [3]:
ecoli_nonsense_error_data

X,Codon,Mean
<chr>,<chr>,<dbl>
GCA,GCA,0.0001747868
GCC,GCC,0.0002856806
GCG,GCG,0.0002191697
GCT,GCT,0.0001000000
TGC,TGC,0.0001000000
TGT,TGT,0.0001658320
GAC,GAC,0.0001000000
GAT,GAT,0.0002260806
GAA,GAA,0.0001000000


In [4]:
# Retrieve Elongation Rates and corresponding Amino acid pero codon
# Elongation rates are stored in file "data_ecoli_shah2010plosgen.csv" under $Rc. Amino acids are store under $AA
ecoli_codon_data = read.csv(file = "/Users/ale/Documents/thesis_codon_bias/Testing_the_TEH/data/data_ecoli_shah2010plosgen.csv", header = TRUE)

In [5]:
source("sequence_conversions.R")

In [6]:
# Assign elongation rates (units: amino acids per second [aa/s]) to a vector
elongation_rates_ecoli <- ecoli_codon_data$Rc
names(elongation_rates_ecoli) <- mrna.string.to.seq.string(ecoli_codon_data$Codon)

In [7]:
# Elongation times = 1/elongation rates (units: seconds per amino acids [s/aa])
ecoli_codon_data$elongation_times_ecoli <- 1 / elongation_rates_ecoli

In [8]:
# Select columns "X...AA", "Codon" and "elongation_times_ecoli"
ecoli_codon_elong_data <- ecoli_codon_data[, c("X...AA", "Codon", "elongation_times_ecoli")]

# Apply the mrna.string.to.seq.string function to rename nucleotides U to T
ecoli_codon_elong_data$Codon <- mrna.string.to.seq.string(ecoli_codon_elong_data$Codon)
ecoli_codon_elong_data

X...AA,Codon,elongation_times_ecoli
<chr>,<chr>,<dbl>
A,GCA,0.04652028
A,GCC,0.03674039
A,GCG,0.07267442
A,GCT,0.04532886
C,TGC,0.13960631
C,TGT,0.21815009
D,GAC,0.04653760
D,GAT,0.07271670
E,GAA,0.03490401


In [9]:
# Get columns "Codon" and "Mean" from ecoli_nonsense_error_data
ecoli_nonsense_error_data <- ecoli_nonsense_error_data[, c("Codon", "Mean")]
ecoli_nonsense_error_data

Codon,Mean
<chr>,<dbl>
GCA,0.0001747868
GCC,0.0002856806
GCG,0.0002191697
GCT,0.0001000000
TGC,0.0001000000
TGT,0.0001658320
GAC,0.0001000000
GAT,0.0002260806
GAA,0.0001000000


In [10]:
# Combine all the data from "ecoli_codon_data" and "ecoli_nonsense_error_data" mapping via column "Codon"
ecoli_codon_complete_data <- merge(ecoli_codon_elong_data, ecoli_nonsense_error_data, by = "Codon")
ecoli_codon_complete_data

Codon,X...AA,elongation_times_ecoli,Mean
<chr>,<chr>,<dbl>,<dbl>
AAA,K,0.02326880,0.0001000000
AAC,N,0.03490401,0.0001000000
AAG,K,0.03635835,0.0001286009
AAT,N,0.05453752,0.0004063484
ACA,U,0.13927577,0.0004459334
ACC,U,0.05367398,0.0001648525
ACG,U,0.05286250,0.0003646348
ACT,U,0.07421701,0.0001000000
AGA,R,0.13952839,0.0008104918


In [11]:
# Rearrange alphabetically by column "X...AA"
ecoli_codon_complete_data <- ecoli_codon_complete_data[order(ecoli_codon_complete_data$X...AA), ]

# Rename columns "X...AA" to "AA" and column "Mean" to "nonsense_error_rates"
colnames(ecoli_codon_complete_data)[colnames(ecoli_codon_complete_data) == "X...AA"] <- "AA"
colnames(ecoli_codon_complete_data)[colnames(ecoli_codon_complete_data) == "Mean"] <- "nonsense_error_rates"
ecoli_codon_complete_data

,Codon,AA,elongation_times_ecoli,nonsense_error_rates
,<chr>,<chr>,<dbl>,<dbl>
37,GCA,A,0.04652028,0.0001747868
38,GCC,A,0.03674039,0.0002856806
39,GCG,A,0.07267442,0.0002191697
40,GCT,A,0.04532886,0.0001000000
55,TGC,C,0.13960631,0.0001000000
57,TGT,C,0.21815009,0.0001658320
34,GAC,D,0.04653760,0.0001000000
36,GAT,D,0.07271670,0.0002260806
33,GAA,E,0.03490401,0.0001000000


In [12]:
# Window over codon, add column with elongation_times_ecoli rank
ecoli_codon_complete_data$elongation_times_ecoli_rank <- ave(ecoli_codon_complete_data$elongation_times_ecoli, ecoli_codon_complete_data$AA, FUN = rank)

# Window over codon, add column with nonsense_error_rates rank
ecoli_codon_complete_data$nonsense_error_rates_rank <- ave(ecoli_codon_complete_data$nonsense_error_rates, ecoli_codon_complete_data$AA, FUN = rank)

# Reorder columns to have elongation_times_ecoli, elongation_times_ecoli_rank, nonsense_error_rates, nonsense_error_rates_rank
ecoli_codon_complete_data <- ecoli_codon_complete_data[, c("Codon", "AA", "elongation_times_ecoli", "elongation_times_ecoli_rank", "nonsense_error_rates", "nonsense_error_rates_rank")]
ecoli_codon_complete_data

,Codon,AA,elongation_times_ecoli,elongation_times_ecoli_rank,nonsense_error_rates,nonsense_error_rates_rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
37,GCA,A,0.04652028,3,0.0001747868,2
38,GCC,A,0.03674039,1,0.0002856806,4
39,GCG,A,0.07267442,4,0.0002191697,3
40,GCT,A,0.04532886,2,0.0001000000,1
55,TGC,C,0.13960631,1,0.0001000000,1
57,TGT,C,0.21815009,2,0.0001658320,2
34,GAC,D,0.04653760,1,0.0001000000,1
36,GAT,D,0.07271670,2,0.0002260806,2
33,GAA,E,0.03490401,1,0.0001000000,1


In [13]:
head(ecoli_codon_complete_data)

,Codon,AA,elongation_times_ecoli,elongation_times_ecoli_rank,nonsense_error_rates,nonsense_error_rates_rank
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
37,GCA,A,0.04652028,3,0.0001747868,2
38,GCC,A,0.03674039,1,0.0002856806,4
39,GCG,A,0.07267442,4,0.0002191697,3
40,GCT,A,0.04532886,2,0.0001000000,1
55,TGC,C,0.13960631,1,0.0001000000,1
57,TGT,C,0.21815009,2,0.0001658320,2


In [14]:
data_gfp_zrinka <- read.csv("/Users/ale/Documents/thesis_codon_bias/Testing_the_TEH/data/data_gfp_zrinka.csv", header = TRUE, sep = ",")

# Keep only 'X...sequence_name...', 'strain...' and 'fluorescence.value..AU.' columns
data_gfp_zrinka <- data_gfp_zrinka[,c(1,2,4)]

# Compute mean fluorescence value for each (sequence, strain) group
mean_fluo <- aggregate(fluorescence.value..AU. ~ X...sequence_name... + strain..., data = data_gfp_zrinka, FUN = mean)
names(mean_fluo) <- c("X...sequence_name...","strain...","mean_fluorescence")

# Add the mean fluorescence value for each row
data_gfp_zrinka <- merge(data_gfp_zrinka, mean_fluo, by = c("X...sequence_name...","strain..."))

In [15]:
# Rename columns in dataframe for better understanding
names(data_gfp_zrinka) <- c("sequence_name", "strain", "value", "mean_fluorescence")

# Remove rows where sequence_name is pET28b_empty (they're controls and carry no info)
data_gfp_zrinka <- data_gfp_zrinka[data_gfp_zrinka$sequence_name != "pET28b_empty",]

In [16]:
# test
data_gfp_zrinka

,sequence_name,strain,value,mean_fluorescence
,<chr>,<chr>,<dbl>,<dbl>
1,V015-wildtype,BL21DE3,16486.75,17098.38
2,V015-wildtype,BL21DE3,16621.75,17098.38
3,V015-wildtype,BL21DE3,17863.75,17098.38
4,V015-wildtype,BL21DE3,13851.75,17098.38
5,V015-wildtype,BL21DE3,19477.00,17098.38
6,V015-wildtype,BL21DE3,19002.00,17098.38
7,V015-wildtype,BL21DE3,19599.00,17098.38
8,V015-wildtype,BL21DE3,13885.00,17098.38
9,V015-wildtype,K12,14853.50,14075.00


In [17]:
# Add sequences to dataframe
# Add sequence_id column: split sequence_name by "-" and keep first element
data_gfp_zrinka$sequence_id <- sapply(strsplit(data_gfp_zrinka$sequence_name, "-"), "[", 1)

# Read sequence constructs file
sequence_constructs <- read.csv("../data/sequence_names_mRNA.csv")

# Add sequence_id column by construct name
sequence_constructs$sequence_id <- sapply(strsplit(sequence_constructs$Construct.name, "_"), "[", 1)

# Add only sequence column from sequence_constructs to data_gfp_zrinka
data_gfp_zrinka <- merge(data_gfp_zrinka, sequence_constructs[,c(4,5)], by = "sequence_id")

In [18]:
# test
data_gfp_zrinka

sequence_id,sequence_name,strain,value,mean_fluorescence,Sequence
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
V015,V015-wildtype,BL21DE3,16486.75,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA
V015,V015-wildtype,BL21DE3,16621.75,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA
V015,V015-wildtype,BL21DE3,17863.75,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA
V015,V015-wildtype,BL21DE3,13851.75,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAG

In [19]:
# Group data by sequence_name, strain, value, mean_fluorescence
data_gfp_zrinka <- aggregate(value ~ sequence_name + strain + mean_fluorescence + Sequence, data = data_gfp_zrinka, FUN = mean)
data_gfp_zrinka

sequence_name,strain,mean_fluorescence,Sequence,value
<chr>,<chr>,<dbl>,<chr>,<dbl>
V015-wildtype,K12,14075.000,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,14075.000
V015-wildtype,BL21DE3,17098.375,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,17098.375
V026-Twist (Variant 4),K12,21145.000,ATGGGAATAATGGAGGCTGAAAGGAAAACAACGGGATGGGCGGCGCGTGACCCATCTGGTATCCTGAGTCCTTATACCTATACGCTGCGTGAAACCGGTCCGGAAGATGTCAATATTCGCATCATCTGTTGTGGTATCTGCCATACCGATTTGCATCAAACCAAAAACGACCTCGGTATGTCCAATTATCCGATGGTTCCGGGTCACGAAGTTGTGGGCGAAGTGGTGGAGGTTGGTTCAGACGTAAGCAAATTCACCGTGGGTGATATTGTTGGCGTGGGCTGCCTGGTTGGGTGCTGCGGTGGCTGCAGCCCGTGTGAACGCGACCTGGAGCAATATTGTCCGAAAAAGATCTGGAGCTATAACGACGTGTACATTAATGGTCAGCCGACTCAGGGAGGCTTCGCGAAGGCTACCGTCGTTCACCAGAAATTCGTGGTAAAAATCCCGGAAGGCATGGCCGTTGAGCAAGCAGCTCCGCTGTTATGCGCGGGTGTTACCGTTTACAGCCCGCTTAGCCACTTTGGTTTGAAGCAGCCAGGCTTAAGAGGTGGTATTCTAGGTTTGGGTGGTGTTGGCCACATGGGCGTGAAGATTGCGAAGGCTATGGGCCATCATGTTACTGTCATCAGCTCCTCCAACAAAAAACGTGAGGAAGCGTTGCAAGACTTGGGCGCAGATGATTACGTGATCGGTTCCGACCAGGCGAAGATGTCTGAGCTGGCGGATAGCCTGGACTACGTCATCGATACCGTCCCGGTGCACCACGCCCTGGAGCCGTACCTGAGCCTGCTGAAGCTGGACGGCAAACTGATTCTGATGGGCGTGATCAACAACCCGTTACAGTTTCTGACCCCGTTGCTCATGCTGGGCCGTAAAGTTATTACCGGTTCGTTCATCGGCAGCATGAAGGAGACGGAAGAGATGCTGGAGTTCTGCAAAGAAAAGGGTCTGTCCAGCATTATTGAAGTCGTGAAGATGGATTACGTGAATACCGCATTTGAACGTCTGGAGAAGAACGACGTGCGTTACCGCTTTGTGGTTGATGTTGAGGGTTCTAACCTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,21145.000
V026-Twist (Variant 4),BL21DE3,30459.375,ATGGGAATAATGGAGGCTGAAAGGAAAACAACGGGATGGGCGGCGCGTGACCCATCTGGTATCCTGAGTCCTTATACCTATACGCTGCGTGAAACCGGTCCGGAAGATGTCAATATTCGCATCATCTGTTGTGGTATCTGCCATACCGATTTGCATCAAACCAAAAACGACCTCGGTATGTCCAATTATCCGATGGTTCCGGGTCACGAAGTTGTGGGCGAAGTGGTGGAGGTTGGTTCAGACGTAAGCAAAT

In [20]:
# Get the row with the highest mean_fluorescence in data_gfp_zrinka
data_gfp_zrinka_max <- data_gfp_zrinka[which.max(data_gfp_zrinka$mean_fluorescence),]
data_gfp_zrinka_max

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
29,V028-NovoPro (Variant 1),BL21DE3,33359.12,ATGGGTATCATGGAAGCTGAACGTAAAACTACCGGTTGGGCTGCACGTGATCCGTCCGGTATTCTGTCTCCGTACACTTACACCCTGCGCGAAACTGGCCCGGAAGATGTGAACATTCGTATCATCTGTTGCGGTATTTGCCACACTGACCTGCACCAGACTAAAAACGACCTGGGCATGTCTAACTATCCGATGGTTCCAGGTCACGAAGTAGTTGGCGAAGTCGTCGAAGTCGGCAGCGACGTTAGCAAGTTCACCGTTGGCGACATCGTGGGTGTAGGCTGCCTGGTTGGTTGTTGTGGCGGTTGTTCCCCGTGTGAGCGTGACCTGGAACAATATTGTCCGAAAAAAATCTGGTCCTACAACGACGTCTATATCAACGGCCAGCCGACCCAGGGTGGTTTCGCAAAAGCTACTGTGGTCCACCAGAAATTCGTAGTTAAAATCCCGGAAGGCATGGCTGTAGAACAGGCTGCTCCACTGCTGTGCGCTGGCGTTACTGTTTACTCCCCGCTGAGCCACTTCGGTCTGAAACAGCCGGGTCTGCGCGGTGGTATTCTGGGTCTGGGTGGTGTGGGTCACATGGGCGTTAAAATCGCCAAAGCAATGGGCCACCACGTGACCGTGATCTCTTCTTCCAATAAGAAACGTGAAGAAGCACTGCAAGATCTGGGTGCAGACGACTACGTTATTGGCTCTGATCAGGCGAAAATGTCTGAGCTGGCTGACTCTCTGGACTATGTAATTGACACTGTGCCGGTACACCACGCACTGGAACCGTACCTGAGCCTGCTGAAACTGGATGGTAAACTGATTCTGATGGGTGTCATCAACAACCCACTGCAATTCCTGACCCCGCTGCTGATGCTGGGTCGTAAAGTTATCACTGGCAGCTTCATTGGCAGCATGAAAGAAACCGAAGAAATGCTGGAATTTTGCAAAGAAAAAGGTCTGTCCTCTATTATCGAAGTTGTCAAGATGGACTACGTAAACACCGCGTTCGAGCGTCTGGAAAAGAACGACGTGCGTTATCGTTTCGTTGTTGATGTTGAAGGTAGCAACCTGGATGCCTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,33359.12


In [21]:
# Get the row with the lowest mean_fluorescence per strain in data_gfp_zrinka
data_gfp_zrinka_min <- data_gfp_zrinka[which.min(data_gfp_zrinka$mean_fluorescence),]
data_gfp_zrinka_min

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
22,V037-RTCIM (Variant 3),BL21DE3,3424.75,ATGGGCATTATGGAAGCGGAACGCAAAACGACGGGTTGGGCGGCTCGCGACCCTTCGGGGATCCTGTCGCCTTACACGTACACGTTACGCGAAACGGGCCCTGAAGACGTGAACATTCGCATCATTTGCTGCGGCATCTGCCACACTGACTTACACCAGACGAAAAACGACTTAGGTATGTCGAACTACCCTATGGTGCCTGGGCACGAAGTGGTGGGGGAAGTGGTGGAAGTGGGCTCGGACGTGAGCAAATTCACTGTGGGGGACATTGTGGGCGTGGGCTGCCTGGTGGGCTGCTGCGGCGGCTGCAGCCCTTGCGAACGCGACTTAGAACAGTACTGCCCTAAAAAAATTTGGAGCTACAACGACGTGTACATCAACGGCCAGCCTACGCAGGGTGGTTTCGCGAAAGCTACTGTTGTGCACCAGAAATTCGTGGTTAAAATTCCTGAAGGCATGGCTGTGGAACAGGCGGCTCCGTTATTATGCGCGGGCGTGACGGTGTACAGCCCTTTAAGCCACTTCGGCTTAAAACAGCCTGGTTTACGCGGCGGCATTTTAGGGTTAGGCGGCGTTGGCCACATGGGCGTGAAAATTGCTAAAGCGATGGGCCACCACGTGACGGTTATTAGCTCGTCGAACAAAAAACGCGAAGAAGCGTTACAGGACTTAGGCGCGGACGACTACGTGATCGGCTCTGACCAGGCTAAAATGAGCGAATTAGCGGACTCCTTAGACTACGTGATTGACACTGTGCCTGTGCACCACGCGTTAGAACCTTACTTATCGTTATTAAAATTAGACGGCAAACTGATTCTGATGGGCGTGATCAACAACCCTTTACAGTTCCTGACGCCTTTATTAATGTTAGGGCGCAAAGTGATTACTGGGAGCTTCATTGGGAGCATGAAAGAAACGGAAGAAATGTTAGAATTCTGCAAAGAAAAAGGCTTAAGCTCCATTATCGAAGTGGTGAAAATGGACTACGTGAACACGGCTTTCGAACGCCTGGAAAAAAACGACGTGCGCTACCGCTTCGTTGTGGACGTTGAAGGCAGCAACCTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,3424.75


In [32]:
# filter out the strian == "BL21DE3" and keep only the strain == "K12"
data_gfp_zrinka_k12 <- data_gfp_zrinka[data_gfp_zrinka$strain == "K12",]
data_gfp_zrinka_k12

# filter out the strian == "K12" and keep only the strain == "BL21DE3"
data_gfp_zrinka_bl21 <- data_gfp_zrinka[data_gfp_zrinka$strain == "BL21DE3",]
data_gfp_zrinka_bl21

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,V015-wildtype,K12,14075.000,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,14075.000
3,V026-Twist (Variant 4),K12,21145.000,ATGGGAATAATGGAGGCTGAAAGGAAAACAACGGGATGGGCGGCGCGTGACCCATCTGGTATCCTGAGTCCTTATACCTATACGCTGCGTGAAACCGGTCCGGAAGATGTCAATATTCGCATCATCTGTTGTGGTATCTGCCATACCGATTTGCATCAAACCAAAAACGACCTCGGTATGTCCAATTATCCGATGGTTCCGGGTCACGAAGTTGTGGGCGAAGTGGTGGAGGTTGGTTCAGACGTAAGCAAATTCACCGTGGGTGATATTGTTGGCGTGGGCTGCCTGGTTGGGTGCTGCGGTGGCTGCAGCCCGTGTGAACGCGACCTGGAGCAATATTGTCCGAAAAAGATCTGGAGCTATAACGACGTGTACATTAATGGTCAGCCGACTCAGGGAGGCTTCGCGAAGGCTACCGTCGTTCACCAGAAATTCGTGGTAAAAATCCCGGAAGGCATGGCCGTTGAGCAAGCAGCTCCGCTGTTATGCGCGGGTGTTACCGTTTACAGCCCGCTTAGCCACTTTGGTTTGAAGCAGCCAGGCTTAAGAGGTGGTATTCTAGGTTTGGGTGGTGTTGGCCACATGGGCGTGAAGATTGCGAAGGCTATGGGCCATCATGTTACTGTCATCAGCTCCTCCAACAAAAAACGTGAGGAAGCGTTGCAAGACTTGGGCGCAGATGATTACGTGATCGGTTCCGACCAGGCGAAGATGTCTGAGCTGGCGGATAGCCTGGACTACGTCATCGATACCGTCCCGGTGCACCACGCCCTGGAGCCGTACCTGAGCCTGCTGAAGCTGGACGGCAAACTGATTCTGATGGGCGTGATCAACAACCCGTTACAGTTTCTGACCCCGTTGCTCATGCTGGGCCGTAAAGTTATTACCGGTTCGTTCATCGGCAGCATGAAGGAGACGGAAGAGATGCTGGAGTTCTGCAAAGAAAAGGGTCTGTCCAGCATTATTGAAGTCGTGAAGATGGATTACGTGAATACCGCATTTGAACGTCTGGAGAAGAACGACGTGCGTTACCGCTTTGTGGTTGATGTTGAGGGTTCTAACCTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,21145.000
6,V025-GenScript (Variant 1),K12,21068.375,ATGGGAATAATGGAGGCTGAAAGGAAAACAACTGGTTGGGCTGCCCGTGATCCGAGCGGTATCCTATCTCCGTATACGTACACTCTGCGTGAGACCGGTCCGGAGGATGTGAATATTCGTATTATTTGCTGCGGCATCTGCCATACCGATTTACACCAAACCAAAAATGACCTGGGCATGTCCAATTATCCGATGGTTCCAGGTCACGAAGTGGTGGGCGAGGTTGTGGAGGTGGGTTCTGACGTGAGCAAGTTCACCGTGGGCGATATTGTTGGCGTTGGCTGCCTGGTTGGTTGTTGTGGTGGCTGCAGCCCGTGTGAACGTGATTTGGAGCAGTACTGCCCGAAAAAAATCTGGTCATATAATGACGTTTATATTAACGGTCAGCCGACCCAGGGTGGTTTTGCGAAGGCTACCGTAGTGCACCAGAAATTCGTGGTCAAGATTCCGGAAGGTATGGCAGTTGAACAAGCGGCTCCGTTGCTGTGTGCGGGTGTGACCGTTTACAGCCCGTTGTCCCATTTTGGTCTGAAGCAGCCAGGTCTGCGCGGTGGTATCCTGGGCCTCGGCGGTGTCGGACACATGGGTGTTAAGATCGCGAAGGCCATGGGCCACCATGTTACGGTGATCAGCAGCTCCAACAAAAAGCGCGAAGAAGCACTGCAAGATCTCGGTGCGGATGACTACGTGATCGGTTCGGACCAGGCGAAGATGAGCGAACTGGCGGACAGCTTGGACTACGTAATTGATACCGTTCCGGTTCACCACGCCTTGGAGCCGTACCTGAGCCTGTTGAAACTGGACGGCAAACTGATTTTAATGGGTGTCATCAACAACCCGCTGCAATTTCTGACCCCGCTTCTGATGCTGGGCAGAAAAGTTATCACGGGCTCCTTCATCGGCAGTATGAAAGAGACAGAAGAGATGCTGGAGTTCTGCAAAGAAAAGGGTTTGTCGAGCATTATCGAAGTTGTTAAGATGGATTACGTGAACACCGCATTTGAACGCCTGGAGAAGAACGACGTGCGTTATCGTTTCGTGGTCGATGTAGAGGGCTCTAACTTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,21068.375
8,V032-IDT (Variant 2),K12,17725.875,ATGGGAATTATGGAAGCAGAACGTAAAACGACTGGGTGGGCAGCTCGTGATCCCTCAGGAATTTTAAGTCCATACACATATACTCTTCGTGAGACTGGGCCAGAAGACGTAAATATTCGCATCATCTGTTGCGGAATCTGTCATACCGACCTTCACCAAACCAAAAACGACTTGGGAATGTCAAACTACCCTATGGTACCCGGCCATGAAGTAGTCGGGGAAGTAGTTGAGGTCGGCTCT

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
2,V015-wildtype,BL21DE3,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,17098.38
4,V026-Twist (Variant 4),BL21DE3,30459.38,ATGGGAATAATGGAGGCTGAAAGGAAAACAACGGGATGGGCGGCGCGTGACCCATCTGGTATCCTGAGTCCTTATACCTATACGCTGCGTGAAACCGGTCCGGAAGATGTCAATATTCGCATCATCTGTTGTGGTATCTGCCATACCGATTTGCATCAAACCAAAAACGACCTCGGTATGTCCAATTATCCGATGGTTCCGGGTCACGAAGTTGTGGGCGAAGTGGTGGAGGTTGGTTCAGACGTAAGCAAATTCACCGTGGGTGATATTGTTGGCGTGGGCTGCCTGGTTGGGTGCTGCGGTGGCTGCAGCCCGTGTGAACGCGACCTGGAGCAATATTGTCCGAAAAAGATCTGGAGCTATAACGACGTGTACATTAATGGTCAGCCGACTCAGGGAGGCTTCGCGAAGGCTACCGTCGTTCACCAGAAATTCGTGGTAAAAATCCCGGAAGGCATGGCCGTTGAGCAAGCAGCTCCGCTGTTATGCGCGGGTGTTACCGTTTACAGCCCGCTTAGCCACTTTGGTTTGAAGCAGCCAGGCTTAAGAGGTGGTATTCTAGGTTTGGGTGGTGTTGGCCACATGGGCGTGAAGATTGCGAAGGCTATGGGCCATCATGTTACTGTCATCAGCTCCTCCAACAAAAAACGTGAGGAAGCGTTGCAAGACTTGGGCGCAGATGATTACGTGATCGGTTCCGACCAGGCGAAGATGTCTGAGCTGGCGGATAGCCTGGACTACGTCATCGATACCGTCCCGGTGCACCACGCCCTGGAGCCGTACCTGAGCCTGCTGAAGCTGGACGGCAAACTGATTCTGATGGGCGTGATCAACAACCCGTTACAGTTTCTGACCCCGTTGCTCATGCTGGGCCGTAAAGTTATTACCGGTTCGTTCATCGGCAGCATGAAGGAGACGGAAGAGATGCTGGAGTTCTGCAAAGAAAAGGGTCTGTCCAGCATTATTGAAGTCGTGAAGATGGATTACGTGAATACCGCATTTGAACGTCTGGAGAAGAACGACGTGCGTTACCGCTTTGTGGTTGATGTTGAGGGTTCTAACCTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,30459.38
5,V027-GenScript (Variant 2),BL21DE3,30932.62,ATGGGAATAATGGAGGCTGAAAGGAAAACAACGGGCTGGGCAGCGCGTGATCCGTCAGGTATTCTGAGCCCGTATACCTATACGCTGCGCGAGACTGGCCCGGAAGATGTTAATATTCGTATTATCTGCTGCGGTATTTGTCATACCGATCTGCATCAGACCAAAAACGACTTGGGCATGTCCAATTATCCGATGGTTCCGGGTCACGAAGTGGTGGGTGAAGTCGTAGAGGTGGGTTCTGATGTCAGCAAATTCACCGTGGGTGACATCGTGGGCGTGGGCTGCCTGGTTGGTTGTTGTGGCGGGTGCAGCCCGTGTGAACGCGACTTAGAGCAGTACTGCCCGAAAAAGATCTGGAGCTATAACGACGTTTACATTAACGGTCAGCCGACCCAGGGTGGTTTTGCGAAGGCTACCGTAGTCCACCAGAAATTCGTTGTAAAGATCCCGGAAGGCATGGCAGTTGAGCAAGCGGCGCCTTTGTTATGCGCGGGTGTGACTGTCTACAGCCCGTTAAGCCATTTTGGTCTGAAACAACCAGGTCTAAGAGGCGGTATCTTGGGACTGGGCGGTGTTGGCCACATGGGCGTGAAGATTGCGAAAGCCATGGGTCACCATGTTACCGTGATTAGCAGCAGCAATAAAAAACGTGAGGAGGCGCTGCAAGATCTCGGCGCTGATGATTACGTGATCGGCTCCGACCAGGCCAAGATGTCCGAGCTGGCTGACAGCTTGGACTACGTTATCGATACCGTTCCAGTTCACCACGCCCTGGAGCCGTACTTGTCCCTGCTGAAGTTGGACGGCAAATTGATTCTGATGGGTGTTATCAACAACCCGTTGCAATTTCTGACCCCGCTGCTTATGCTGGGTCGTAAAGTTATTACGGGCTCGTTCATCGGCAGTATGAAAGAAACCGAAGAAATGCTGGAGTTCTGCAAGGAGAAGGGTCTGAGCTCTATCATCGAAGTGGTGAAGATGGATTATGTTAATACCGCATTTGAACGCCTGGAGAAGAACGACGTTCGTTACCGTTTCGTGGTGGACGTCGAGGGTTCTAACCTGGACGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,30932.62
7,V025-GenScript (Variant 1),BL21DE3,29605.50,ATGGGAATAATGGAGGCTGAAAGGAAAACAACTGGTTGGGCTGCCCGTGATCCGAGCGGTATCCTATCTCCGTATACGTACACTCTGCGTGAGACCGGTCCGGAGGATGTGAATATTCGTATTATTTGCTGCGGCATCTGCCATACCGATTTACACCAAACCAAAAATGACCTGGGCATGTCCAATTATCCGATGGTTCCAGGTCACGAAGTGGTGGGCGAGGTT

In [23]:
# Get the row with the highest mean_fluorescence for data_gfp_zrinka_k12
data_gfp_zrinka_k12_max <- data_gfp_zrinka_k12[which.max(data_gfp_zrinka_k12$mean_fluorescence),]
data_gfp_zrinka_k12_max

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
27,V018-JCAT,K12,26028.25,ATGGGTATCATGGAAGCTGAACGTAAAACCACCGGTTGGGCTGCTCGTGACCCGTCTGGTATCCTGTCTCCGTACACCTACACCCTGCGTGAAACCGGTCCGGAAGACGTTAACATCCGTATCATCTGCTGCGGTATCTGCCACACCGACCTGCACCAGACCAAAAACGACCTGGGTATGTCTAACTACCCGATGGTTCCGGGTCACGAAGTTGTTGGTGAAGTTGTTGAAGTTGGTTCTGACGTTTCTAAATTCACCGTTGGTGACATCGTTGGTGTTGGTTGCCTGGTTGGTTGCTGCGGTGGTTGCTCTCCGTGCGAACGTGACCTGGAACAGTACTGCCCGAAAAAAATCTGGTCTTACAACGACGTTTACATCAACGGTCAGCCGACCCAGGGTGGTTTCGCTAAAGCTACCGTTGTTCACCAGAAATTCGTTGTTAAAATCCCGGAAGGTATGGCTGTTGAACAGGCTGCTCCGCTGCTGTGCGCTGGTGTTACCGTTTACTCTCCGCTGTCTCACTTCGGTCTGAAACAGCCGGGTCTGCGTGGTGGTATCCTGGGTCTGGGTGGTGTTGGTCACATGGGTGTTAAAATCGCTAAAGCTATGGGTCACCACGTTACCGTTATCTCTTCTTCTAACAAAAAACGTGAAGAAGCTCTGCAGGACCTGGGTGCTGACGACTACGTTATCGGTTCTGACCAGGCTAAAATGTCTGAACTGGCTGACTCTCTGGACTACGTTATCGACACCGTTCCGGTTCACCACGCTCTGGAACCGTACCTGTCTCTGCTGAAACTGGACGGTAAACTGATCCTGATGGGTGTTATCAACAACCCGCTGCAGTTCCTGACCCCGCTGCTGATGCTGGGTCGTAAAGTTATCACCGGTTCTTTCATCGGTTCTATGAAAGAAACCGAAGAAATGCTGGAATTTTGCAAAGAAAAAGGTCTGTCTTCTATCATCGAAGTTGTTAAAATGGACTACGTTAACACCGCTTTCGAACGTCTGGAAAAAAACGACGTTCGTTACCGTTTCGTTGTTGACGTTGAAGGTTCTAACCTGGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,26028.25


In [24]:
# Get the row with the lowest mean_fluorescence for data_gfp_zrinka_k12
data_gfp_zrinka_k12_min <- data_gfp_zrinka_k12[which.min(data_gfp_zrinka_k12$mean_fluorescence),]
data_gfp_zrinka_k12_min

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
35,V021-Twist (Variant 3),K12,9991.125,ATGGGTATTATGGAAGCTGAAAGAAAGACGACGGGTTGGGCGGCGCGTGATCCGTCGGGTATTCTTAGCCCATATACCTATACGCTCCGTGAAACAGGTCCCGAAGACGTCAATATTCGCATTATCTGTTGCGGGATTTGTCATACGGACCTGCACCAGACCAAGAACGACCTGGGTATGAGCAACTATCCAATGGTCCCGGGTCACGAGGTTGTGGGCGAGGTGGTTGAAGTCGGTTCTGACGTTAGTAAATTTACAGTGGGTGATATTGTCGGTGTGGGGTGTTTGGTCGGTTGCTGTGGTGGATGCTCTCCGTGTGAACGCGACCTTGAGCAATACTGCCCTAAGAAGATATGGAGTTATAACGACGTGTATATTAACGGCCAGCCCACCCAGGGTGGTTTTGCAAAGGCGACGGTAGTACATCAGAAATTCGTTGTAAAGATACCCGAGGGTATGGCCGTCGAACAAGCCGCTCCTCTTTTATGTGCAGGGGTTACGGTTTATTCACCTTTGTCTCATTTCGGCTTAAAGCAGCCTGGTTTACGTGGCGGCATTCTGGGCCTGGGCGGCGTTGGCCACATGGGAGTAAAGATTGCTAAGGCGATGGGACATCACGTTACAGTTATTTCTAGCAGCAATAAGAAACGTGAGGAAGCTCTGCAGGACTTGGGTGCCGACGACTATGTTATAGGTAGCGATCAGGCAAAGATGTCTGAGCTGGCAGACAGTCTCGACTATGTCATCGATACAGTTCCAGTGCACCACGCGTTGGAACCCTACCTGTCATTGCTGAAATTGGACGGAAAGCTGATCTTAATGGGTGTCATTAATAACCCGCTGCAATTCTTAACACCGTTATTAATGCTGGGTCGAAAGGTCATTACCGGTAGTTTTATTGGTTCTATGAAAGAAACCGAAGAAATGCTGGAATTTTGCAAGGAGAAAGGACTGAGCTCCATCATTGAGGTAGTCAAAATGGACTACGTGAATACGGCCTTCGAACGCCTGGAAAAGAATGACGTCCGCTACCGGTTTGTGGTCGACGTTGAGGGCAGTAACCTGGATGCGTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATAA,9991.125


In [33]:
# Get the wildtype sequence from data_gfp_zrinka_k12, sequence_name == "V015-wildtype"
data_gfp_zrinka_k12_wt <- data_gfp_zrinka_k12[data_gfp_zrinka_k12$sequence_name == "V015-wildtype",]
data_gfp_zrinka_k12_wt

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,V015-wildtype,K12,14075,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,14075


In [34]:
# Get the wildtype sequence from data_gfp_zrinka_bl21, sequence_name == "V015-wildtype"
data_gfp_zrinka_bl21_wt <- data_gfp_zrinka_bl21[data_gfp_zrinka_bl21$sequence_name == "V015-wildtype",]
data_gfp_zrinka_bl21_wt

,sequence_name,strain,mean_fluorescence,Sequence,value
,<chr>,<chr>,<dbl>,<chr>,<dbl>
2,V015-wildtype,BL21DE3,17098.38,ATGGGAATAATGGAGGCAGAGAGGAAAACAACAGGCTGGGCTGCCAGAGACCCATCTGGCATCCTCTCTCCTTACACTTACACTCTTAGAGAGACTGGACCAGAGGATGTGAACATAAGAATCATTTGCTGTGGAATCTGCCACACCGATCTTCATCAAACTAAAAATGATCTTGGCATGTCTAATTACCCCATGGTTCCTGGGCATGAAGTGGTAGGGGAAGTAGTGGAGGTGGGATCAGATGTGAGCAAGTTCACCGTAGGGGACATAGTTGGAGTTGGTTGCCTCGTTGGATGTTGCGGAGGTTGTAGCCCCTGCGAGAGAGATCTGGAACAGTATTGTCCAAAGAAGATTTGGAGCTACAATGATGTTTACATCAATGGTCAACCTACACAAGGCGGCTTCGCTAAAGCCACCGTCGTTCACCAAAAGTTTGTGGTCAAGATTCCAGAAGGAATGGCGGTTGAGCAGGCTGCGCCGCTACTGTGCGCTGGTGTGACTGTGTACAGTCCACTGAGCCACTTTGGTCTGAAACAACCAGGCCTAAGAGGAGGTATACTAGGGTTAGGTGGAGTCGGTCACATGGGTGTGAAAATAGCCAAAGCAATGGGTCACCATGTGACTGTCATAAGCTCATCAAACAAGAAGAGAGAAGAGGCATTGCAAGATCTTGGAGCTGATGATTACGTGATCGGTTCCGACCAAGCGAAGATGAGCGAATTGGCTGATTCGTTGGATTACGTAATTGACACGGTGCCTGTTCATCATGCACTTGAGCCATATTTGTCTCTGCTTAAGCTTGATGGTAAACTCATTCTCATGGGAGTTATCAACAATCCATTACAGTTTCTCACTCCTCTGCTTATGCTTGGGAGGAAAGTGATAACGGGGAGCTTCATAGGGAGCATGAAGGAGACAGAGGAGATGCTTGAGTTCTGTAAAGAAAAGGGTTTGAGTTCGATTATCGAAGTTGTGAAGATGGATTATGTTAACACTGCGTTTGAGAGACTTGAGAAGAACGATGTGCGTTATAGGTTCGTCGTTGATGTCGAAGGAAGCAATCTCGACGCTTTAATTGGCTCCGATGGAGGGTCTGGTGGCGGATCAACAAGTCGTGACCACATGGTCCTTCATGAGTACGTAAATGCTGCTGGGATTACATGA,17098.38


In [27]:
codon.analysis <- function(dataframe_cand, ecoli_codon_complete_data, verbose = TRUE){

    # retrieve all the codons of the sequence
    # function "seq.string.to.cod.string" is sourced from "sequence_conversions.R".
    codons_of_sequence <- seq.string.to.cod.string(dataframe_cand$Sequence)
    # remove the last item in codons_of_sequence because it is the stop codon
    codons_of_sequence_exc_stop_codon <- codons_of_sequence[-length(codons_of_sequence)]
    # count frequency of each codon in codons_of_sequence
    codon_freq <- table(codons_of_sequence_exc_stop_codon)

    # print the sequence_name and strain of the seq_cand
    if(verbose){
      cat("The sequence name is", dataframe_cand$sequence_name, '\n')
      cat("The strain is", dataframe_cand$strain, '\n')
    }
    # print number of different codons used in the sequence
    if(verbose){
      cat("The number of different codons used in the sequence is", length(codon_freq), '\n')
    }

    # convert codon_freq to a dataframe
    codon_freq <- as.data.frame(codon_freq)
    # rename the columns of codon_freq
    colnames(codon_freq) <- c("Codon", "Frequency")

    # Merge codon_freq and ecoli_codon_complete_data
    ecoli_codon_analysis <- merge(ecoli_codon_complete_data, codon_freq, by = "Codon")

    # Sort the dataframe by Frequency in descending order
    ecoli_codon_analysis <- ecoli_codon_analysis[order(-ecoli_codon_analysis$Frequency),]

    # Erase the index
    rownames(ecoli_codon_analysis) <- NULL

    # Reorder the columns of ecoli_codon_analysis "AA", "Codon", "Frequency", "elongation_times_ecoli", "elongation_times_ecoli_rank", "nonsense_error_rates_ecoli", "nonsense_error_rates_rank"
    ecoli_codon_analysis <- ecoli_codon_analysis[,c(2,1,7,3,5,4,6)]

    # calculate and print number of codons where "nonsense_error_rates_rank" == 1 is used in sequence
    if(verbose){
      cat("The number of most accurate codons used in the sequence is", length(which(ecoli_codon_analysis$nonsense_error_rates_rank == 1, )), '\n')
    }
    # calculate and print number of codons where "elongation_times_ecoli_rank" == 1 is used in sequence
    if(verbose){
      cat("The number of fastest codons used in the sequence is", length(which(ecoli_codon_analysis$elongation_times_ecoli_rank == 1, )), '\n')
    }

    # Print the dataframe
    return(ecoli_codon_analysis)
}

# Test, passing the arguments "data_gfp_zrinka_max" and "ecoli_codon_complete_data"
codon.analysis(data_gfp_zrinka_max, ecoli_codon_complete_data)

The sequence name is V028-NovoPro (Variant 1) 
The strain is BL21DE3 
The number of different codons used in the sequence is 49 
The number of most accurate codons used in the sequence is 21 
The number of fastest codons used in the sequence is 19 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
L,CTG,33,0.03008152,0.0001000000,1,1
G,GGT,25,0.04417351,0.0001000000,2,1
E,GAA,21,0.03490401,0.0001000000,1,1
K,AAA,18,0.02326880,0.0001000000,1,1
G,GGC,17,0.03034717,0.0001679261,1,2
V,GTT,15,0.03262004,0.0001000000,3,1
M,ATG,14,0.01745170,0.0001000000,1,1
D,GAC,14,0.04653760,0.0001000000,1,1
P,CCG,13,0.08509914,0.0001000000,1,1


In [28]:
codon.analysis(data_gfp_zrinka_max, ecoli_codon_complete_data)

The sequence name is V028-NovoPro (Variant 1) 
The strain is BL21DE3 
The number of different codons used in the sequence is 49 
The number of most accurate codons used in the sequence is 21 
The number of fastest codons used in the sequence is 19 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
L,CTG,33,0.03008152,0.0001000000,1,1
G,GGT,25,0.04417351,0.0001000000,2,1
E,GAA,21,0.03490401,0.0001000000,1,1
K,AAA,18,0.02326880,0.0001000000,1,1
G,GGC,17,0.03034717,0.0001679261,1,2
V,GTT,15,0.03262004,0.0001000000,3,1
M,ATG,14,0.01745170,0.0001000000,1,1
D,GAC,14,0.04653760,0.0001000000,1,1
P,CCG,13,0.08509914,0.0001000000,1,1


In [29]:
codon.analysis(data_gfp_zrinka_min, ecoli_codon_complete_data)

The sequence name is V037-RTCIM (Variant 3) 
The strain is BL21DE3 
The number of different codons used in the sequence is 43 
The number of most accurate codons used in the sequence is 21 
The number of fastest codons used in the sequence is 19 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
V,GTG,31,0.04361289,0.0002272305,4,3
G,GGC,28,0.03034717,0.0001679261,1,2
L,TTA,27,0.13945056,0.0005234102,6,5
E,GAA,24,0.03490401,0.0001000000,1,1
K,AAA,22,0.02326880,0.0001000000,1,1
D,GAC,21,0.04653760,0.0001000000,1,1
I,ATT,16,0.07254262,0.0002641397,3,2
P,CCT,15,0.11253657,0.0002854809,3,3
M,ATG,14,0.01745170,0.0001000000,1,1


In [30]:
codon.analysis(data_gfp_zrinka_k12_max, ecoli_codon_complete_data)

The sequence name is V018-JCAT 
The strain is K12 
The number of different codons used in the sequence is 37 
The number of most accurate codons used in the sequence is 19 
The number of fastest codons used in the sequence is 17 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
G,GGT,40,0.04417351,0.0001000000,2,1
V,GTT,37,0.03262004,0.0001000000,3,1
L,CTG,33,0.03008152,0.0001000000,1,1
E,GAA,24,0.03490401,0.0001000000,1,1
K,AAA,22,0.02326880,0.0001000000,1,1
S,TCT,22,0.07423905,0.0001000000,2,1
I,ATC,21,0.04646624,0.0001000000,2,1
D,GAC,21,0.04653760,0.0001000000,1,1
A,GCT,20,0.04532886,0.0001000000,2,1


In [31]:
codon.analysis(data_gfp_zrinka_k12_min, ecoli_codon_complete_data)

The sequence name is V021-Twist (Variant 3) 
The strain is K12 
The number of different codons used in the sequence is 58 
The number of most accurate codons used in the sequence is 20 
The number of fastest codons used in the sequence is 21 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
G,GGT,22,0.04417351,0.0001000000,2,1
I,ATT,16,0.07254262,0.0002641397,3,2
D,GAC,16,0.04653760,0.0001000000,1,1
K,AAG,15,0.03635835,0.0001286009,2,2
L,CTG,15,0.03008152,0.0001000000,1,1
E,GAA,15,0.03490401,0.0001000000,1,1
M,ATG,14,0.01745170,0.0001000000,1,1
V,GTC,13,0.02792282,0.0002789281,2,4
G,GGC,12,0.03034717,0.0001679261,1,2


In [35]:
codon.analysis(data_gfp_zrinka_bl21_wt, ecoli_codon_complete_data)

The sequence name is V015-wildtype 
The strain is BL21DE3 
The number of different codons used in the sequence is 58 
The number of most accurate codons used in the sequence is 21 
The number of fastest codons used in the sequence is 21 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
E,GAG,16,0.05453752,0.0001757650,2,2
D,GAT,16,0.07271670,0.0002260806,2,2
G,GGA,15,0.13921760,0.0005818305,4,4
V,GTG,15,0.04361289,0.0002272305,4,3
M,ATG,14,0.01745170,0.0001000000,1,1
G,GGT,14,0.04417351,0.0001000000,2,1
K,AAG,13,0.03635835,0.0001286009,2,2
L,CTT,12,0.11238481,0.0003419059,4,3
V,GTT,12,0.03262004,0.0001000000,3,1


In [36]:
codon.analysis(data_gfp_zrinka_k12_wt, ecoli_codon_complete_data)

The sequence name is V015-wildtype 
The strain is K12 
The number of different codons used in the sequence is 58 
The number of most accurate codons used in the sequence is 21 
The number of fastest codons used in the sequence is 21 


AA,Codon,Frequency,elongation_times_ecoli,nonsense_error_rates,elongation_times_ecoli_rank,nonsense_error_rates_rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
E,GAG,16,0.05453752,0.0001757650,2,2
D,GAT,16,0.07271670,0.0002260806,2,2
G,GGA,15,0.13921760,0.0005818305,4,4
V,GTG,15,0.04361289,0.0002272305,4,3
M,ATG,14,0.01745170,0.0001000000,1,1
G,GGT,14,0.04417351,0.0001000000,2,1
K,AAG,13,0.03635835,0.0001286009,2,2
L,CTT,12,0.11238481,0.0003419059,4,3
V,GTT,12,0.03262004,0.0001000000,3,1
